# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [5]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Create test text_dict

In [6]:
test_text1 = """Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p

Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p
 = 0.37) and increased ratings of pain intensity (by 0.7 ± 0.7 cm; P = 0.004; η2p
 = 0.55) and unpleasantness (by 0.7 ± 0.9 cm; P = 0.02; η2p
 = 0.40); these results were not influenced by menstrual phase. Water ingestion reduced thirst perception (visual analog scale, by 2.3 ± 0.9 cm; P < 0.001, η2p
 = 0.88) but did not reduce pain sensitivity. Therefore, hypohydration increases pain sensitivity in women with no influence of menstrual phase.
"""

test_text2 = """Daily Energy Expenditure through the Human Life Course

Total daily energy expenditure (“total expenditure”, MJ/d) reflects daily energy needs and is a critical variable in human health and physiology, yet it is unclear how daily expenditure changes over the life course. Here, we analyze a large, globally diverse database of total expenditure measured by the doubly labeled water method for males and females aged 8 days to 95 yr. We show that total expenditure is strongly related to fat free mass in a power-law manner and identify four distinct metabolic life stages. Fat free mass-adjusted daily expenditure accelerates rapidly in neonates (0-1yr) to ~46% above adult values at ~1 yr, declines slowly throughout childhood and adolescence (1-20 yr) to adult levels at ~20 yr, remains stable in adulthood (20-60 yr) even during pregnancy, and declines in older adults (60+ yr). These changes in total expenditure shed new light on human development and aging and should help shape nutrition and health strategies across the lifespan.
"""

test_text_dict = create_text_dict([test_text1, test_text2])
test_text_dict.keys()

dict_keys([1, 2])

# Create prompts_df

In [8]:
from itertools import product
prompts_dict = dict()
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    # "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    # "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
    columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

prompts_df = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])


prompts_df['prompt'] = prompts_df.apply(
    lambda row: f"{row['prep_step']} {row['task part 1']}.", 
    axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
# prompts_df['relevance'] = prompts_df.apply(
#     lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
#     axis=1) 
prompts_dict[experiment_num] = prompts_df
prompts_df

,prep_step,task part 1,prompt
0,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post.


# Iteration 1

In [9]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() # use an ordered dictionary instead of a regular dictionary
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        print('temp_qna_dict index:', pd.DataFrame(temp_qna_dict[key]).index)
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = [prompt for prompt in simplify_prompts if len(prompt) >1]
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        for summary_key in summary_keys:
            print(f'\tSimplifying {summary_key}...')
            for index, prompt in enumerate(simplify_prompts):
                print(f'\t...Prompt: {prompt}')
                simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
        try:
            temp_qna_chain_dict = pd.concat([qna_chain_dict[summary_iteration_id], pd.DataFrame(simple_summaries_dict)])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 1
n_choices = 2
qna_dict = dict()
qna_chain_dict = dict()
chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)



**Text #1 prompt #0 of 0**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	**API request failed**
	...Success!
temp_qna_dict index: RangeIndex(start=0, stop=2, step=1)
**Text #2 prompt #0 of 0**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
temp_qna_dict index: Index(['article_title', 'model', 'prompt', 'response_01', 'response_02',
       'system_role', 'text'],
      dtype='object')
Error concatenating DataFrames


In [10]:
qna_dict[iteration_id]

,0
0,"{'article_title': 'Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p', 'system_role': 'You are an expert at science communication.', 'prompt': 'Take the key points and numerical descriptors to Summarize for a LinkedIn post.', 'model': 'gpt-3.5-turbo'}"
1,<summarization.Chatbot object at 0x000001EEF6D555B0>
prep_step,Take the key points and numerical descriptors to
task part 1,Summarize for a LinkedIn post
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.


# *End of Page*